In [1]:
!pip install ndx-pose

In [ ]:
!gdown --fuzzy https://drive.google.com/file/d/1nN9XWL2L-ZeSb5rqu38gC28EOVlYuaOq/view?usp=drive_link

Downloading...
From (original): https://drive.google.com/uc?id=1nN9XWL2L-ZeSb5rqu38gC28EOVlYuaOq
From (redirected): https://drive.google.com/uc?id=1nN9XWL2L-ZeSb5rqu38gC28EOVlYuaOq&confirm=t&uuid=f6e015d8-9625-4b60-a583-959012baf7ac
To: /content/save_data_AVG.mat
100% 467M/467M [00:06<00:00, 75.4MB/s]


In [2]:
from scipy.io import loadmat
import datetime
import numpy as np
from pynwb import NWBFile, NWBHDF5IO
from ndx_pose import PoseEstimationSeries, PoseEstimation


mat = loadmat("save_data_AVG.mat")
print(mat.keys())
print(mat["data"].shape)
print(mat["p_max"].shape)

dict_keys(['__header__', '__version__', '__globals__', 'pred', 'data', 'p_max', 'sampleID', 'metadata'])
(360000, 3, 23)
(360000, 23)


In [3]:
for n in mat["metadata"].dtype.names:
    x = mat["metadata"][n]
    print(f"{n}: {x}")

augment_brightness: [[array([[0]])]]
augment_hue: [[array([[0]])]]
com_exp: [[array([[(array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_21_1/20201221_163226_Label3D_dannce.mat'],
                dtype='<U115'),)                                                                                                         ]],
        dtype=[('label3d_file', 'O')])                                                                                                      ]]
com_file: [[array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_22_1/COM/predict00/com3d.mat'],
        dtype='<U104')                                                                                               ]]
com_predict_dir: [[array(['./COM/predict00'], dtype='<U15')]]
com_predict_weights: [[array(['/n/holylfs02/LABS/olveczky_lab/Everyone/dannce_rig/dannce_ephys/art/2020_12_21_1/COM/train00/fullmodel_weights/fullmodel_end.hdf5'],
        dtype='<U129')      

In [17]:
nwb_filename = 'save_data_AVG_2.nwb'

# Pull out core data.
trx = mat["pred"]
print("trx shape", trx.shape)
trx_new = trx[0:1, :, :]
trx_new[0,:,0] = [0,0,-.25]

confidences = mat["p_max"]
print(confidences.shape)
n_frames, xyz, n_nodes = trx.shape

# TODO: replace this with the actual node names (not in the MAT file metadata?)
node_names = [f"part_{i:02}" for i in range(n_nodes)]
node_names = [f"part_0"]

fps = 30
#timestamps = np.arange(n_frames) / 30  # a timestamp for every frame
timestamps = np.arange(999) / 30  # a timestamp for every frame

nwbfile = NWBFile(
    session_description='session_description',
    identifier='identifier',
    session_start_time=datetime.datetime.now(datetime.timezone.utc)
)

pose_estimation_series = []
for i, node_name in enumerate(node_names):
    print("node_name ", node_name)
    #data = trx[0:999, :, i]  # num_frames x (x, y, z)
    data = trx_new[0:1, :, i]
    confidence = confidences[0:1, i]  # a confidence value for every node on every frame

    node_series = PoseEstimationSeries(
        name=node_name,
        description='',
        data=data,
        unit='pixels',
        reference_frame='',
        timestamps=timestamps,
        confidence=confidence,
        confidence_definition='',
    )

    pose_estimation_series.append(node_series)


pe = PoseEstimation(
    pose_estimation_series=pose_estimation_series,
    description='',
    # original_videos=[''],
    # labeled_videos=[''],
    # dimensions=[],
    # scorer='',
    source_software='DANNCE',
    source_software_version='',
    nodes=node_names,
    edges=[],  # TODO: Add edges for viz?
)

behavior_pm = nwbfile.create_processing_module(
    name='behavior',
    description='processed behavioral data'
)
behavior_pm.add(pe)

with NWBHDF5IO(nwb_filename, mode='w') as io:
    io.write(nwbfile)


trx shape (360000, 3, 23)
(360000, 23)
node_name  part_0


/opt/conda/envs/stac-mjx-env/lib/python3.11/site-packages/pynwb/base.py:203: UserWarning: PoseEstimationSeries 'part_0': Length of data does not match length of timestamps. Your data may be transposed. Time should be on the 0th dimension
  warn("%s '%s': Length of data does not match length of timestamps. Your data may be transposed. "


In [18]:
# Test reading the data back in and converting to concrete numpy array.
with NWBHDF5IO(nwb_filename, mode='r', load_namespaces=True) as io:
    read_nwbfile = io.read()
    #print((type(read_nwbfile)))
    read_nwbfile.stimulus
    read_pe = read_nwbfile.processing['behavior']['PoseEstimation']

    node_names = read_pe.nodes[:].tolist()

    trx = []
    for node_name in node_names:
        #print("node_name: ", node_name)
        trx.append(read_pe[node_name].data[:])

    trx = np.stack(trx, axis=-1)

trx.shape

/opt/conda/envs/stac-mjx-env/lib/python3.11/site-packages/pynwb/base.py:203: UserWarning: PoseEstimationSeries 'part_0': Length of data does not match length of timestamps. Your data may be transposed. Time should be on the 0th dimension
  warn("%s '%s': Length of data does not match length of timestamps. Your data may be transposed. "


(1, 3, 1)

In [19]:
import h5py

# Path to the NWB file
file_path = nwb_filename

# Open the NWB file
with h5py.File(file_path, 'r') as f:
    # Traverse the file and print the size of each dataset
    def print_dataset_info(name, obj):
        if isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name}")
            print(f"  Size: {obj.size}")
            print(f"  Shape: {obj.shape}")
            print(f"  Data Type: {obj.dtype}")

    f.visititems(print_dataset_info)


Dataset: file_create_date
  Size: 1
  Shape: (1,)
  Data Type: object
Dataset: identifier
  Size: 1
  Shape: ()
  Data Type: object
Dataset: processing/behavior/PoseEstimation/description
  Size: 1
  Shape: ()
  Data Type: object
Dataset: processing/behavior/PoseEstimation/edges
  Size: 0
  Shape: (0,)
  Data Type: uint8
Dataset: processing/behavior/PoseEstimation/nodes
  Size: 1
  Shape: (1,)
  Data Type: object
Dataset: processing/behavior/PoseEstimation/part_0/confidence
  Size: 1
  Shape: (1,)
  Data Type: float64
Dataset: processing/behavior/PoseEstimation/part_0/data
  Size: 3
  Shape: (1, 3)
  Data Type: float64
Dataset: processing/behavior/PoseEstimation/part_0/reference_frame
  Size: 1
  Shape: ()
  Data Type: object
Dataset: processing/behavior/PoseEstimation/part_0/timestamps
  Size: 999
  Shape: (999,)
  Data Type: float64
Dataset: processing/behavior/PoseEstimation/source_software
  Size: 1
  Shape: ()
  Data Type: object
Dataset: session_description
  Size: 1
  Shape: ()
